author: Diogo Silva

In [1]:
import numpy as np
import seaborn as snb
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [5]:
from MyML.cluster.K_Means3 import K_Means
import MyML.helper.partition as partMod

In [ ]:
n_samples = 1e4
n_samples = int(n_samples)
n_features = 2
n_clusters = 6

In [ ]:
g1 = np.random.normal(loc=(0,0), scale=1, size=(n_samples/n_clusters, n_features))
g2 = np.random.normal(loc=(10,5), scale=0.5, size=(n_samples/n_clusters, n_features))
g3 = np.random.normal(loc=(0,8), scale=0.1, size=(n_samples/n_clusters, n_features))
g4 = np.random.normal(loc=(-10,5), scale=(0.25,1), size=(n_samples/n_clusters, n_features))
g5 = np.random.normal(loc=(5,10), scale=0.5, size=(n_samples/n_clusters, n_features))
g6 = np.random.normal(loc=(10,-5), scale=0.5, size=(n_samples/n_clusters, n_features))

data = np.vstack((g1,g2,g3,g4,g5,g6))

plt.plot(data[:,0],data[:,1],'.')

In [ ]:
del g1, g2, g3, g4, g5, g6, data

In [56]:
foldername = "/home/diogoaos/QCThesis/datasets/gaussmix1e4/"
dataname = "data.csv"

In [57]:
np.savetxt(foldername + dataname, data, delimiter = ",")

In [7]:
data = np.genfromtxt(foldername + dataname, delimiter = ",", dtype = np.float32)

In [62]:
n_partitions = 100
ensemble_clusters = [np.sqrt(n_samples)/4, np.sqrt(n_samples)/2]
ensemble_clusters = map(int, ensemble_clusters)
generator = K_Means(cuda_mem="manual")

In [63]:
%time partMod.generateEnsembleToFiles(foldername, data, generator, n_clusters=ensemble_clusters, npartitions=n_partitions)

WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted to float32. TODO: accept other formats
WARNING DATA DUPLICATION: data converted

# Build co-association matrix

## Load ensemble

In [4]:
import MyML.cluster.eac as eac

In [5]:
filename=foldername+"part0.csv"

NameError: name 'foldername' is not defined

In [160]:
filelist = !ls $foldername

In [162]:
prefix = ''
filelist = [foldername + prefix + fname for fname in filelist if "part" in fname]

In [ ]:
ensembe = partMod.loadEnsembleFromFiles(filelist=filelist)

In [256]:
max_val = 0
for clust in ensemble[0]:
    temp = clust.max()
    if temp > max_val:
        max_val = temp
n_samples = max_val + 1

In [7]:
fullEAC = eac.EAC(n_samples = n_samples, mat_sparse = False)

In [259]:
%time fullEAC.fit(ensemble, assoc_mode="full")

CPU times: user 3.38 s, sys: 0 ns, total: 3.38 s
Wall time: 3.38 s


Total time = 3.37s

In [271]:
coassoc = np.zeros((n_samples, n_samples), dtype=np.int8)

In [272]:
%time eac.update_coassoc_with_ensemble(coassoc, ensemble)

CPU times: user 975 ms, sys: 12.3 ms, total: 987 ms
Wall time: 978 ms


Total time = 987 ms

In [280]:
protEAC = eac.EAC(n_samples = n_samples, mat_sparse = False)

In [281]:
%time protEAC.fit(ensemble, assoc_mode="prot", prot_mode="random", nprot=int(np.sqrt(n_samples)))

CPU times: user 421 ms, sys: 3 µs, total: 421 ms
Wall time: 429 ms


Total time = 421 ms

In [193]:
sparseEAC = eac.EAC(n_samples = n_samples, mat_sparse = True)

In [194]:
%time sparseEAC.fit(ensemble, assoc_mode="full")

CPU times: user 13min 20s, sys: 726 ms, total: 13min 21s
Wall time: 13min 21s


Total time = 13min 21s

In [202]:
from scipy.sparse import csr_matrix

In [282]:
%time coassoc_sp = csr_matrix(coassoc)

CPU times: user 1.02 s, sys: 0 ns, total: 1.02 s
Wall time: 1.02 s


Total time to build sparse from full = 1.02s

In [283]:
print "density=",coassoc_sp.nnz/(n_samples**2.0)

density= 0.156158101496


In [288]:
coassoc_sp.indptr[-1]

15603320

In [319]:
1e5*1e4/1024/1024

953.67431640625

In [320]:
%%time
sp0=csr_matrix(np.random.randint(0, 2, (100000,10000)))
sp1=csr_matrix(np.random.randint(0, 2, (100000,10000)))

CPU times: user 57.9 s, sys: 27.9 s, total: 1min 25s
Wall time: 4min 17s


In [322]:
sp0.nnz/(1.0*np.prod(sp0.shape))

0.500006773

In [323]:
%time sp2 = sp0 + sp1

CPU times: user 5.1 s, sys: 1.02 s, total: 6.12 s
Wall time: 8.68 s


# SL-HAC

In [8]:
import MyML.graph.mst as mst
import MyML.graph.connected_components as ccomps

/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/cuda/decorators.py:106: UserWarning: autojit is deprecated and will be removed in a future release. Use jit instead.
  warn('autojit is deprecated and will be removed in a future release. Use jit instead.')


## Get graph

In [ ]:
fullEAC._lifetime_clustering(coassoc, method='single')

In [ ]:
np.random.randint